In [3]:
# add src to path
from pyprojroot import here
import sys

sys.path.append(str(here("src")))

import json
import os
from pathlib import Path
import numpy as np
from typing import Dict, List, Any


def is_numeric(value: Any) -> bool:
    """Check if a value is numeric."""
    try:
        float(value)
        return True
    except (ValueError, TypeError):
        return False


def flatten_dict(d: Dict, parent_key: str = "", sep: str = "_") -> Dict:
    """Flatten a nested dictionary."""
    items = []
    for k, v in d.items():
        new_key = f"{parent_key}{sep}{k}" if parent_key else k
        if isinstance(v, dict):
            items.extend(flatten_dict(v, new_key, sep=sep).items())
        else:
            items.append((new_key, v))
    return dict(items)


def compute_averages(file_path: Path) -> Dict[str, float]:
    """Compute averages for numeric columns in a JSONL file."""
    data = []
    with open(file_path, "r") as f:
        for line in f:
            # Flatten the nested dictionary
            flat_data = flatten_dict(json.loads(line))
            data.append(flat_data)

    if not data:
        return {}

    # Get all unique keys from all entries
    all_keys = set()
    for entry in data:
        all_keys.update(entry.keys())

    # Initialize results dictionary
    results = {}

    # Compute averages for numeric columns
    for key in all_keys:
        values = [entry.get(key) for entry in data if key in entry]
        numeric_values = [v for v in values if is_numeric(v)]

        if numeric_values:
            results[key] = np.mean(numeric_values)

    return results


# Get the results directory path
results_dir = Path("../results")

# Process each JSONL file
for file_path in results_dir.glob("*.jsonl"):
    print(f"\nProcessing {file_path.name}:")
    print("-" * 50)

    averages = compute_averages(file_path)

    if averages:
        print("Column averages:")
        for key, value in sorted(averages.items()):
            print(f"{key}: {value:.4f}")
    else:
        print("No numeric columns found or file is empty.")


Processing classification_bms_correlations.jsonl:
--------------------------------------------------
Column averages:
average_results_agreement_id: 0.9239
average_results_agreement_ood: 0.8674
average_results_total_variation_distance_id: 0.0524
average_results_total_variation_distance_ood: 0.0947
context_length: 256.0000
correlation_bms_relative_distance_id: 0.9781
correlation_bms_relative_distance_ood: 0.9785
mlp_expansion_factor: 4.1250
num_dims: 10.0000

Processing linear-regression_bms_correlations.jsonl:
--------------------------------------------------
Column averages:
average_results_r_squared_id: 0.9658
average_results_r_squared_ood: 0.2594
context_length: 37.3333
correlation_bms_relative_distance_id: 0.9872
correlation_bms_relative_distance_ood: 0.9733
mlp_expansion_factor: 4.1667
num_dims: 12.0000

Processing categorical-sequence_bms_correlations.jsonl:
--------------------------------------------------
Column averages:
average_results_spearman_correlation_id: 0.9684
averag